In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [47]:
#Create empty lists to store the data we are about to collect from the website
#We would like to collect the names of the cars, the mileage, the price, the rating, the number of reviews the car received

car_name = []
car_mileage = []
car_price = []

In [48]:
#We need to create a variable to store the url of the website, I'm calling this variable website_url. We then need to send a request to the website
#and scrape multiple pages

for i in range(1,11):
    website_url ="https://www.truecar.com/used-cars-for-sale/listings/location-boston-ma/?mmt[]=toyota_tacoma&mmt[]=toyota_4runner&page=" + str(i)
    #make request to the website
    web_request = requests.get(website_url, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36'})
    
    # Create an object to store the html elements of the request
    soup = BeautifulSoup(web_request.content,'html.parser')
    
    # Create a list that contains the results of the web page
    results_list = soup.find_all('div', {'class':'mt-1 flex w-full items-start p-2 text-left'})
    
    for i in results_list:
    
        #get the names of the cars. If the name of the car is not present, append 'n/a' to the list
        try:
            car_name.append(i.find('h2').get_text())
        except:
            car_name.append('n/a')

        #get the mileage of the cars
        try:
            car_mileage.append(i.find('div', {'data-test': 'vehicleMileage'}).get_text())
        except:
            car_mileage.append('n/a')

        #get the price of the car
        try:
            car_price.append(i.find('span', {'data-test': 'vehicleCardPriceLabelAmount'}).get_text())
        except:
            car_price.append('n/a')


In [49]:
# Create a DataFrame

car_info = pd.DataFrame({'Name': car_name, 'Mileage': car_mileage, 'Price': car_price})

In [50]:
#print the DataFrame
car_info

,Name,Mileage,Price
0,Used 2020 ToyotaTacoma SR5 Access Cab 6' Bed V...,55k mi,"$29,990"
1,Used 2023 ToyotaTacoma SR Double Cab 5' Bed V6...,21k mi,"$29,996"
2,Used 2023 ToyotaTacoma TRD Sport Double Cab 5'...,23k mi,"$36,400"
3,Used 2018 ToyotaTacoma SR Access Cab 6.1' Bed ...,129k mi,"$14,883"
4,Used 2013 Toyota4Runner Limited V6 4WD,160k mi,"$16,995"
...,...,...,...
298,Used 2020 Toyota4Runner TRD Off Road Premium 4WD,89k mi,"$32,895"
299,Used 2021 ToyotaTacoma SR5 Double Cab 5' Bed V...,40k mi,"$31,998"
300,Used 2024 ToyotaTacoma TRD Off Road Double Cab...,339 mi,"$48,991"
301,Used 2014 ToyotaTacoma Access Cab V6 4WD Autom...,71k mi,"$28,998"


In [51]:
#Let's clean the data frame

#The first thing we should do is to edit the Reviews column

car_info['Mileage'] = car_info['Mileage'].apply(lambda x:x.replace('mi', '')) #remove 'mi' from the mileage column
car_info['Mileage'] = car_info['Mileage'].apply(lambda x:x.replace('k', '000')) #remove 'k' from the mileage column
car_info['Price'] = car_info['Price'].apply(lambda x:x.replace('$', '')) # remove the $ from the Price column
car_info['Price'] = car_info['Price'].apply(lambda x:x.replace(',', '')) #remove the ',' from the Price column

In [52]:
car_info

,Name,Mileage,Price
0,Used 2020 ToyotaTacoma SR5 Access Cab 6' Bed V...,55000,29990
1,Used 2023 ToyotaTacoma SR Double Cab 5' Bed V6...,21000,29996
2,Used 2023 ToyotaTacoma TRD Sport Double Cab 5'...,23000,36400
3,Used 2018 ToyotaTacoma SR Access Cab 6.1' Bed ...,129000,14883
4,Used 2013 Toyota4Runner Limited V6 4WD,160000,16995
...,...,...,...
298,Used 2020 Toyota4Runner TRD Off Road Premium 4WD,89000,32895
299,Used 2021 ToyotaTacoma SR5 Double Cab 5' Bed V...,40000,31998
300,Used 2024 ToyotaTacoma TRD Off Road Double Cab...,339,48991
301,Used 2014 ToyotaTacoma Access Cab V6 4WD Autom...,71000,28998


In [53]:
#The Name column has a lot of important data on the Year, Company, Vehicle make and model.
# Use regex to split the 'Name' column into 4 parts: Stock type, Year, Brand, and the rest
#(\w+): Matches the stock type (e.g., "Used").
#(\d{4}): Matches the year (e.g., "2021").
#([A-Za-z]+): Matches the brand (e.g., "Toyota").
#(.*): Captures the rest of the string (e.g., "Tacoma SR5 Double Cab 5' Bed V6 4WD Automatic").

car_info[['Stock type', 'Year', 'Brand', 'Rest']] = car_info['Name'].str.extract(r'(\w+)\s(\d{4})\s([A-Za-z]+)(.*)')


In [54]:
car_info

,Name,Mileage,Price,Stock type,Year,Brand,Rest
0,Used 2020 ToyotaTacoma SR5 Access Cab 6' Bed V...,55000,29990,Used,2020,ToyotaTacoma,SR5 Access Cab 6' Bed V6 4WD Automatic
1,Used 2023 ToyotaTacoma SR Double Cab 5' Bed V6...,21000,29996,Used,2023,ToyotaTacoma,SR Double Cab 5' Bed V6 4WD Automatic
2,Used 2023 ToyotaTacoma TRD Sport Double Cab 5'...,23000,36400,Used,2023,ToyotaTacoma,TRD Sport Double Cab 5' Bed V6 4WD Manual
3,Used 2018 ToyotaTacoma SR Access Cab 6.1' Bed ...,129000,14883,Used,2018,ToyotaTacoma,SR Access Cab 6.1' Bed I4 RWD Automatic
4,Used 2013 Toyota4Runner Limited V6 4WD,160000,16995,Used,2013,Toyota,4Runner Limited V6 4WD
...,...,...,...,...,...,...,...
298,Used 2020 Toyota4Runner TRD Off Road Premium 4WD,89000,32895,Used,2020,Toyota,4Runner TRD Off Road Premium 4WD
299,Used 2021 ToyotaTacoma SR5 Double Cab 5' Bed V...,40000,31998,Used,2021,ToyotaTacoma,SR5 Double Cab 5' Bed V6 2WD Automatic
300,Used 2024 ToyotaTacoma TRD Off Road Double Cab...,339,48991,Used,2024,ToyotaTacoma,TRD Off Road Double Cab 5' Bed 4WD Automatic
301,Used 2014 ToyotaTacoma Access Cab V6 4WD Autom...,71000,28998,Used,2014,ToyotaTacoma,Access Cab V6 4WD Automatic


In [55]:
# For my example, the car name and model are displayed in the brand column. I will use regex to extract the Brand and Model
#NOTE: This may not be the case for you, if it isn't so not run this cell. I suggest you look at your data

#We use the regular expression r'([A-Za-z]+)([A-Z][a-zA-Z]*)' to capture:
#([A-Za-z]+): The first part of the string (brand), which is made up of any sequence of letters.
#([A-Z][a-zA-Z]*): The second part (model), which starts with a capital letter followed by more letters.

car_info[['Make', 'Model']] = car_info['Brand'].str.extract(r'([A-Za-z]+)([A-Z][a-zA-Z]*)')

car_info

,Name,Mileage,Price,Stock type,Year,Brand,Rest,Make,Model
0,Used 2020 ToyotaTacoma SR5 Access Cab 6' Bed V...,55000,29990,Used,2020,ToyotaTacoma,SR5 Access Cab 6' Bed V6 4WD Automatic,Toyota,Tacoma
1,Used 2023 ToyotaTacoma SR Double Cab 5' Bed V6...,21000,29996,Used,2023,ToyotaTacoma,SR Double Cab 5' Bed V6 4WD Automatic,Toyota,Tacoma
2,Used 2023 ToyotaTacoma TRD Sport Double Cab 5'...,23000,36400,Used,2023,ToyotaTacoma,TRD Sport Double Cab 5' Bed V6 4WD Manual,Toyota,Tacoma
3,Used 2018 ToyotaTacoma SR Access Cab 6.1' Bed ...,129000,14883,Used,2018,ToyotaTacoma,SR Access Cab 6.1' Bed I4 RWD Automatic,Toyota,Tacoma
4,Used 2013 Toyota4Runner Limited V6 4WD,160000,16995,Used,2013,Toyota,4Runner Limited V6 4WD,NaN,NaN
...,...,...,...,...,...,...,...,...,...
298,Used 2020 Toyota4Runner TRD Off Road Premium 4WD,89000,32895,Used,2020,Toyota,4Runner TRD Off Road Premium 4WD,NaN,NaN
299,Used 2021 ToyotaTacoma SR5 Double Cab 5' Bed V...,40000,31998,Used,2021,ToyotaTacoma,SR5 Double Cab 5' Bed V6 2WD Automatic,Toyota,Tacoma
300,Used 2024 ToyotaTacoma TRD Off Road Double Cab...,339,48991,Used,2024,ToyotaTacoma,TRD Off Road Double Cab 5' Bed 4WD Automatic,Toyota,Tacoma
301,Used 2014 ToyotaTacoma Access Cab V6 4WD Autom...,71000,28998,Used,2014,ToyotaTacoma,Access Cab V6 4WD Automatic,Toyota,Tacoma


In [56]:
#For my example, there are a few NaNs in the Make and Model columns, I need to replace these with the values Toyota, Tacoma

# Fill NaN values in the 'Brand' column with 'Toyota'
car_info['Make'].fillna('Toyota', inplace=True)

# Fill NaN values in the 'Model' column with 'Tacoma'
car_info['Model'].fillna('Tacoma', inplace=True)

# Display the DataFrame
print(car_info)

                                                  Name  Mileage  Price  \
0    Used 2020 ToyotaTacoma SR5 Access Cab 6' Bed V...   55000   29990   
1    Used 2023 ToyotaTacoma SR Double Cab 5' Bed V6...   21000   29996   
2    Used 2023 ToyotaTacoma TRD Sport Double Cab 5'...   23000   36400   
3    Used 2018 ToyotaTacoma SR Access Cab 6.1' Bed ...  129000   14883   
4               Used 2013 Toyota4Runner Limited V6 4WD  160000   16995   
..                                                 ...      ...    ...   
298   Used 2020 Toyota4Runner TRD Off Road Premium 4WD   89000   32895   
299  Used 2021 ToyotaTacoma SR5 Double Cab 5' Bed V...   40000   31998   
300  Used 2024 ToyotaTacoma TRD Off Road Double Cab...     339   48991   
301  Used 2014 ToyotaTacoma Access Cab V6 4WD Autom...   71000   28998   
302  Used 2023 ToyotaTacoma SR5 Double Cab 5' Bed V...   28000   38970   

    Stock type  Year         Brand  \
0         Used  2020  ToyotaTacoma   
1         Used  2023  ToyotaTacoma 

In [57]:
# Drop the column Name and its no longer needed
car_info = car_info.drop(["Name"], axis = 1)
car_info = car_info.drop(["Rest"], axis = 1)
car_info = car_info.drop(["Brand"], axis = 1)

In [58]:
car_info

,Mileage,Price,Stock type,Year,Make,Model
0,55000,29990,Used,2020,Toyota,Tacoma
1,21000,29996,Used,2023,Toyota,Tacoma
2,23000,36400,Used,2023,Toyota,Tacoma
3,129000,14883,Used,2018,Toyota,Tacoma
4,160000,16995,Used,2013,Toyota,Tacoma
...,...,...,...,...,...,...
298,89000,32895,Used,2020,Toyota,Tacoma
299,40000,31998,Used,2021,Toyota,Tacoma
300,339,48991,Used,2024,Toyota,Tacoma
301,71000,28998,Used,2014,Toyota,Tacoma


In [ ]:
#You can export the DataFrame to excel if you choose to proceed with excel
car_info.to_excel('multiple_pages.xlsx', index = False)

In [ ]:
#You can export the DataFrame to a csv file if needed
car_info.to_csv('multiple_pages.csv', index = False)